# related papers
http://journals.sagepub.com/doi/pdf/10.1177/0081175015576601

http://www.lifescienceglobal.com/pms/index.php/ijcs/article/view/5052/2853

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2770506

https://www.ijdp.org/article/S0955-3959(17)30288-8/abstract

http://article.socialsciencesjournal.org/pdf/10.11648.j.ss.20180702.15.pdf

https://www.sciencedirect.com/science/article/pii/S0091743503002330

https://www.sciencedirect.com/science/article/pii/S0013935109000498

https://onlinelibrary.wiley.com/doi/abs/10.1111/ecpo.12102

http://journals.sagepub.com/doi/abs/10.1177/0003122416663494

http://journals.sagepub.com/doi/pdf/10.1177/0022427806298356

https://waset.org/abstracts/environmental-and-ecological-engineering/76986

http://www.mdpi.com/2220-9964/7/2/43

http://discovery.ucl.ac.uk/11291/1/11291.pdf (hot spot report)

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gp

import datetime
from dateutil.relativedelta import relativedelta


from src import constants as C
from src.data_prep import prep_data_from_raw, prep_911, prep_crime

In [2]:
d911 = prep_911(by_category=False, gpdf=True, coords_series=False)
d911.geometry = d911.buffer(30)
d911.reset_index(inplace=True)
d911.crs

{'init': 'epsg:3559', 'no_defs': True}

In [3]:
crimes = prep_crime(path='data/open-baltimore/clean/crimes-dev-set.csv',by_category=False, gpdf=True, coords_series=False)
crimes.reset_index(inplace=True)
crimes.geometry = crimes.buffer(30)

In [4]:
# crimes = prep_data_from_raw('data/open-baltimore/raw/BPD_Part_1_Victim_Based_Crime_Data.csv', 
#                  col_lon='Longitude', col_lat='Latitude', col_date='CrimeDate', to_epsg=3559)

# crimes= crimes.loc['2015-01-01':]
# crimes['geometry'] = crimes.Coords.apply(lambda x: Point(*x))
# crimes = gp.GeoDataFrame(crimes)
# crimes.geometry = crimes.buffer(30)
# crimes.reset_index(inplace=True)
# crimes.crs = {'init': 'epsg:3559', 'no_defs': True}

In [5]:
ym_crime = set(crimes.Date.apply(lambda x: (x.year, x.month)).unique())
ym_911 = set(d911.Date.apply(lambda x: (x.year, x.month)).unique())
yms = list(ym_crime & ym_911)

In [6]:
pairs = []
for y, m in yms:
    date = datetime.datetime(year=y, month=m, day=1)
    start = date - datetime.timedelta(days=1)
    end = date + relativedelta(months=+1)
    slice_crimes = crimes[(crimes.Date>=start) & (crimes.Date<=end)]
    slice_d911 = d911[(d911.Date>=start) & (d911.Date<=end)]
    joined = gp.sjoin(slice_d911[['Date', 'geometry']], slice_crimes[['Date', 'geometry']])
    pairs.append(
        joined[joined.Date_left == joined.Date_right].reset_index()[['index', 'index_right']].rename(
            columns={'index': 'i911', 'index_right': 'icrime'})    
    )

pairs = pd.concat(pairs, ignore_index=True)

D:\ProgramFiles\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2652: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [7]:
crimes[(crimes.Date>='2015-01-01') & (crimes.Date<='2017-01-01')].shape

(92820, 17)

In [8]:
pairs.groupby('icrime').size().shape

(40206,)

In [9]:
pairs.groupby('i911').size().sort_values(ascending=False)

i911
436077    11
436152    11
436230    11
436196    11
435844    11
436171    11
436284    11
436080    11
436116    11
436136    11
114159     7
114247     7
114249     7
114450     7
114343     7
365345     6
584391     6
584384     6
46395      6
215997     6
95289      6
113471     6
95215      6
71576      6
127676     6
127677     6
392015     6
216003     6
113164     6
112933     6
          ..
415938     1
415934     1
416231     1
416235     1
416527     1
416238     1
416513     1
416511     1
416507     1
416492     1
416457     1
416429     1
416419     1
416412     1
416400     1
416358     1
416353     1
416346     1
416345     1
416342     1
416337     1
416334     1
416309     1
416306     1
416302     1
416270     1
416262     1
416259     1
416251     1
58         1
Length: 60161, dtype: int64

In [10]:
idx_crimes = pairs.icrime.unique()
idx_911 = pairs.i911.unique()

In [75]:
%matplotlib inline

In [47]:
# duplicates in crime
crimes.loc[pairs[pairs.i911==609242].icrime]

,Date,ori_index,Time,CrimeCode,Location,Description,Weapon,Post,District,Neighborhood,Longitude,Latitude,Premise,Category,In/Outside,Coords,geometry
179877,2016-11-30,61079,18:00:00,9S,2900 GARRISON BLVD,SHOOTING,FIREARM,641.0,NORTHWESTERN,Hanlon-Longwood,-76.67729,39.32049,Public Are,shooting,O,"(427828.0384116637, 183634.42648913254)","POLYGON ((427858.0384116637 183634.4264891325,..."
179878,2016-11-30,61075,18:00:00,1F,2900 GARRISON BLVD,HOMICIDE,FIREARM,641.0,NORTHWESTERN,Hanlon-Longwood,-76.67729,39.32049,Public Are,homicide,O,"(427828.0384116637, 183634.42648913254)","POLYGON ((427858.0384116637 183634.4264891325,..."
179886,2016-11-30,61089,18:00:00,4A,2900 GARRISN BLVD,AGG. ASSAULT,FIREARM,641.0,NORTHWESTERN,Hanlon-Longwood,-76.67729,39.32049,OTHER - OU,assault,O,"(427828.0384116637, 183634.42648913254)","POLYGON ((427858.0384116637 183634.4264891325,..."
179878,2016-11-30,61075,18:00:00,1F,2900 GARRISON BLVD,HOMICIDE,FIREARM,641.0,NORTHWESTERN,Hanlon-Longwood,-76.67729,39.32049,Public Are,homicide,O,"(427828.0384116637, 183634.42648913254)","POLYGON ((427858.0384116637 183634.4264891325,..."
179877,2016-11-30,61079,18:00:00,9S,2900 GARRISON BLVD,SHOOTING,FIREARM,641.0,NORTHWESTERN,Hanlon-Longwood,-76.67729,39.32049,Public Are,shooting,O,"(427828.0384116637, 183634.42648913254)","POLYGON ((427858.0384116637 183634.4264891325,..."
179886,2016-11-30,61089,18:00:00,4A,2900 GARRISN BLVD,AGG. ASSAULT,FIREARM,641.0,NORTHWESTERN,Hanlon-Longwood,-76.67729,39.32049,OTHER - OU,assault,O,"(427828.0384116637, 183634.42648913254)","POLYGON ((427858.0384116637 183634.4264891325,..."


In [11]:

from importlib import reload
import src.vis; reload(src.vis)
import src.vis.map; reload(src.vis.map)
from src.vis.map import marker_cluster, marker_cluster_fast


In [17]:
from wKit.vis.map import marker_cluster_fast

In [12]:
target_c = crimes.loc[idx_crimes]
target_9 = d911.loc[idx_911]

In [13]:
name_location = {
    'crimes': list(zip(target_c.Latitude.tolist(), target_c.Longitude.tolist())),
    '911': list(zip(target_9.Latitude.tolist(), target_9.Longitude.tolist()))
}

In [20]:
marker_cluster_fast(name_location, lonlat=False, filename='tmp/crime_911.html')